To Do:
- create downloadable csv for each jurisdiction

In [1]:
import pandas as pd
import urllib.request, json 
from urllib.request import urlopen
import re
from bs4 import BeautifulSoup
import PyPDF2
import os
from os import walk
import os.path
from os import path
import numpy as np
#!pip install py-readability-metrics
from readability import Readability
#!python -m nltk.downloader punkt

import time
from datetime import date

import spacy
from sklearn.preprocessing import normalize

from joblib import dump, load

In [2]:
nlp = spacy.load('en_core_web_lg')

In [3]:
def vectorize(text):
    output = nlp(str(text)).vector   
    return output

def norm(row):
    try:
        matrix = row.reshape(1,-1).astype(np.float64)
        return normalize(matrix, axis=1, norm='l1')[0]
    except Exception as e: 
        print("===================")
        print(row)
        print("-------------------")
        print(e)
        print("===================")
        #return np.NaN

In [25]:
included_fields = load('../data/processed/ML/norm_fields/included_fields.joblib') 
jurisdictions = load('../data/processed/ML/norm_fields/jurisdictions.joblib') 
groups = load('../data/processed/ML/norm_fields/groups.joblib') 
clf_field_names = load('../data/processed/ML/norm_fields/clf_field_names.joblib') 

def normalize_name(jur,group,n,per,last_field,this_field):
    
    # Add hard coded conversions maybe by calling a function
    # if returns 0 then fail over to ML or otherway around poor prob -> check hard-coded

    if this_field not in included_fields:
        this_field = reCase(this_field)

        out_put = re.sub("^(My\s)?Name$","users1_name",this_field, flags=re.IGNORECASE)
        out_put = re.sub("^Printed Name\s?\d*$","users1_name",out_put, flags=re.IGNORECASE)  
        out_put = re.sub("^(DOB|Date of Birth|Birthday)$","users1_birthdate",out_put, flags=re.IGNORECASE)

        out_put = re.sub("^(Street )?Address$","users1_address_line_one",out_put, flags=re.IGNORECASE)
        out_put = re.sub("^City State Zip$","users1_address_line_two",out_put, flags=re.IGNORECASE)
        out_put = re.sub("^City$","users1_address_city",out_put, flags=re.IGNORECASE)
        out_put = re.sub("^Zip( Code)?$","users1_address_zip",out_put, flags=re.IGNORECASE)

        out_put = re.sub("^Phone$","users1_phone_number",out_put, flags=re.IGNORECASE)
        out_put = re.sub("^Email$","users1_email",out_put, flags=re.IGNORECASE)

        out_put = re.sub("^(Court\s)?Case\s?(No|Number)?\s?A?$","docket_number",out_put, flags=re.IGNORECASE)
        out_put = re.sub("^File\s?(No|Number)?\s?A?$","docket_number",out_put, flags=re.IGNORECASE)

        out_put = re.sub("^Date\s?\d*$","signature_date",out_put, flags=re.IGNORECASE)

        conf = 1.0

        if out_put==this_field:
            params = []
            for item in jurisdictions:
                if jur== item:
                    params.append(1)
                else:
                    params.append(0)
            for item in groups:
                if group== item:
                    params.append(1)
                else:
                    params.append(0)
            params.append(n)
            params.append(per)
            for vec in norm(vectorize(this_field)):
                params.append(vec)
            #for vec in norm(vectorize(last_field)):
            #    params.append(vec)

            for item in included_fields:
                if last_field==item:
                    params.append(1)
                else:
                    params.append(0)

            pred = clf_field_names.predict([params])
            prob = clf_field_names.predict_proba([params])

            conf = prob[0].tolist()[prob[0].tolist().index(max(prob[0].tolist()))]
            out_put = pred[0]
            
    else:
        out_put = this_field
        conf = 1
            
    if out_put in included_fields:
        if conf >= 0:
            return "*"+out_put+"| was <i>%s</i> (%.2f conf)"%(this_field,conf) #, conf
        else:
            return this_field+" (predicted <i>%s</i> w/ %.2f conf)"%(out_put,conf)
    else:
        return this_field+" (predicted <i>%s</i> w/ %.2f conf)"%(out_put,conf)


In [26]:
def reCase(text):
    output = re.sub("(\w|\d)(_|-)(\w|\d)","\\1 \\3",text.strip())
    output = re.sub("([a-z])([A-Z]|\d)","\\1 \\2",output)
    output = re.sub("(\d)([A-Z]|[a-z])","\\1 \\2",output)
    return output

In [27]:
def read_pdf (file):
    try:
        pdfFile = PyPDF2.PdfFileReader(open(file, "rb"))
        if pdfFile.isEncrypted:
            try:
                pdfFile.decrypt('')
                #print ('File Decrypted (PyPDF2)')
            except:
                #
                #
                # This didn't go so well on my Windows box so I just ran this in the pdf folder's cmd:
                # for %f in (*.*) do copy %f temp.pdf /Y && "C:\Program Files (x86)\qpdf-8.0.2\bin\qpdf.exe" --password="" --decrypt temp.pdf %f
                #
                #
                #
                
                command="cp "+file+" tmp/temp.pdf; qpdf --password='' --decrypt tmp/temp.pdf "+file
                os.system(command)
                #print ('File Decrypted (qpdf)')
                #re-open the decrypted file
                pdfFile = PyPDF2.PdfFileReader(open(file, "rb"))
        text = ""
        for page in pdfFile.pages:
            text = text + " " + page.extractText()
        return text
    except:
        return ""


In [28]:
def parse_form(fileloc,title=None,jur=None,cat=None,normalize=1):
    f = PyPDF2.PdfFileReader(fileloc)
    npages = f.getNumPages()
    ff = f.getFields()
    if ff:
        fields = list(ff.keys())
    else:
        fields = []
    f_per_page = len(fields)/npages
    text = read_pdf(fileloc)
    try:
        readbility = int(Readability(text).flesch_kincaid().grade_level)
    except:
        readbility = None
    
    if title is None:
        title = reCase(re.search("(.*)\n",text).group(1).strip())
        
    if normalize==1:
        i = 0 
        length = len(fields)
        last = "null"
        new_fields = []
        for field in fields:
            #print(jur,cat,i,i/length,last,field)
            new_fields.append(normalize_name(jur,cat,i,i/length,last,field))
            last = field
    else:
        new_fields = fields
    
    stats = {
            "title":title,
            "category":cat,
            "pages":npages,
            "reading grade level": readbility,
            "avg fields per page": f_per_page,
            "fields":new_fields
            }    
    return stats

In [29]:
parse_form("../data/processed/www.utcourts.gov/forms/a39365b4ef6ec7c699a7dbdde772f787a911746cb66f8d319d495b3b.pdf",title=None,jur="UT",cat=None,normalize=1)

{'title': 'Rev. May 12, 2009',
 'category': None,
 'pages': 4,
 'reading grade level': 12,
 'avg fields per page': 30.5,
 'fields': ['undefined (predicted <i>_nonstandard_</i> w/ 1.00 conf)',
  '*users1_name| was <i>Name</i> (1.00 conf)',
  '*users1_address_line_one| was <i>Address</i> (1.00 conf)',
  'Court Decision 1 (predicted <i>_nonstandard_</i> w/ 1.00 conf)',
  'Court Decision 2 (predicted <i>_nonstandard_</i> w/ 1.00 conf)',
  '*users1_address_city| was <i>City</i> (1.00 conf)',
  'Date of denial (predicted <i>_nonstandard_</i> w/ 1.00 conf)',
  'undefined 2 (predicted <i>_nonstandard_</i> w/ 1.00 conf)',
  'FOR WAIVER OF COURT FEES (predicted <i>_nonstandard_</i> w/ 1.00 conf)',
  'vs (predicted <i>_nonstandard_</i> w/ 1.00 conf)',
  'Appellate Case No (predicted <i>_nonstandard_</i> w/ 1.00 conf)',
  '*docket_number| was <i>Case No</i> (1.00 conf)',
  'I was born on (predicted <i>_nonstandard_</i> w/ 1.00 conf)',
  'If I am applying for my child my childs name is (predicted <

In [30]:
files_df_ut = pd.read_csv("../data/raw/www.utcourts.gov/form_data.csv")
files_df_mi = pd.read_csv("../data/raw/www.courts.michigan.gov/form_data.csv")

df = pd.concat([files_df_ut,files_df_mi],ignore_index=True)

In [31]:
df["pages"] = ""
df["fields"] = ""
df["f_per_p"] = ""
df["reading"] = ""

for index,row in df.iterrows():
    if row["pages"] == "":
        try:
            stats = parse_form("../data/processed/"+row["source"]+"/forms/"+row["id"]+".pdf",row["title"],row["jurisdiction"],row["group"],1)
            df.at[index, 'pages'] = stats["pages"]
            df.at[index, 'fields'] = stats["fields"]
            df.at[index, 'f_per_p'] = stats["avg fields per page"]
            df.at[index, 'reading'] = stats["reading grade level"]
            #print(index)
        except:
            print("error: "+"../data/raw/"+row["source"]+"/forms/"+row["id"]+".pdf")
        
df

id jurisdiction  \
0     04b3a0734774c02edf8eb9056d23954aa38e96c77c3392...           UT   
1     6e420f1b3575cfd8ef94b71977da9e38252e3395a78439...           UT   
2     2532cd2b6d3aaff8c47726a0abd168fb4e5cdb4977c065...           UT   
3     f6a6814890f21c11524d5785d772272916ff95909b7dba...           UT   
4     be656a9a361db7c3532026b0bae372e704beb160fce37e...           UT   
...                                                 ...          ...   
1523  31f0385280ba8218dbace7693ea04977c1cfa3278ec4f8...           MI   
1524  2a0847b00aa3dcf37dff8c1b2c3d4dcb3247445678cda1...           MI   
1525  07e5f95f3f4042ecd5b58fcf8a141e8931ab84124a6c19...           MI   
1526  15f70625faa5b1a09b1839d4003d806921983fea9a8b65...           MI   
1527  ebf739852305fe98fe474c1f8e064bf3a8f58ee73cf558...           MI   

                       source  \
0            www.utcourts.gov   
1            www.utcourts.gov   
2            www.utcourts.gov   
3            www.utcourts.gov   
4            www.utcourts.gov   
...                       ...   
1523  www.courts.michigan.gov   
1524  www.courts.michigan.gov   
1525  www.courts.michigan.gov   
1526  www.courts.michigan.gov   
1527  www.courts.michigan.gov   

                                                  title  \
0     Community Service Worksheet Third District Juv...   
1     Third District Juvenile Court Work Program Ref...   
2     Utah State District Juvenile Court Probation O...   
3     Adoptee's Consent to Adoption and Waiver of Ri...   
4                                   Adoption Agreement    
...                                                 ...   
1523  Claim of Appeal of Right/Request to Withdraw A...   
1524                 Default Judgment, Civil Infraction   
1525                         Judgment, Civil Infraction   
1526  Motion to Set Aside Default Judgment and Order...   
1527                 Notice to Appear, Civil Infraction   

                                                 group  \
0     3rd District Juvenile Court: Forms and Pamphlets   
1     3rd District Juvenile Court: Forms and Pamphlets   
2     3rd District Juvenile Court: Forms and Pamphlets   
3                           Adopting a Minor Stepchild   
4                           Adopting a Minor Stepchild   
...                                                ...   
1523                                  civil infraction   
1524                                  civil infraction   
1525                                  civil infraction   
1526                                  civil infraction   
1527                                  civil infraction   

                                                    url  \
0     https://www.utcourts.gov/courts/juv/juvsites/3...   
1     https://www.utcourts.gov/courts/juv/juvsites/3...   
2     https://www.utcourts.gov/courts/juv/juvsites/3...   
3     https://www.utcourts.gov/howto/family/adoption...   
4     https://www.utcourts.gov/howto/family/adoption...   
...                                                 ...   
1523  https://www.courts.michigan.gov/4a2ca8/siteass...   
1524  https://www.courts.michigan.gov/4a7328/siteass...   
1525  https://www.courts.michigan.gov/4a7336/siteass...   
1526  https://www.courts.michigan.gov/4a2cb4/siteass...   
1527  https://www.courts.michigan.gov/4a2cf6/siteass...   

                                               filename  downloaded pages  \
0     COMMUNITY%20SERVICE%20WORKSHEET-FRONT%20AND%20...  2021-11-11     2   
1                        Work_Crew_Application-2007.pdf  2021-11-11     2   
2                Probation%20Order%20revised.050502.pdf  2021-11-11     5   
3                                02_Consent_Adoptee.pdf  2021-11-11     3   
4                                      08_Agreement.pdf  2021-11-11     1   
...                                                 ...         ...   ...   
1523                                          cia05.pdf  2021-11-11     1   
1524                                          cia07.pdf  202

In [32]:
df.to_csv("../data/processed/form_data.csv", index=False, encoding="utf-8")    

In [24]:
os.system('cp ../data/processed/www.utcourts.gov/forms/* ../data/processed/forms/')
os.system('cp ../data/processed/www.courts.michigan.gov/forms/* ../data/processed/forms/')

0